## Set-up

In [ ]:
import tifffile
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
import gc
from matplotlib import colormaps as cm

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import utils
import track

### Hyper parameters

In [ ]:
loadstar_weight_name = "loadstar_first"
magik_weight_name = "MAGIK_MP_MPN"

# Experiment parameters
tiff_path = r'/home/gideon/Documents/PhD/Projects/KAW/data_from_VE/Experiment-16031.tif' # Change this to where you have the tiff file
output_path = r'/home/gideon/Documents/PhD/Projects/KAW/output' # Change this to where you want the output to be saved
pages_batch_size = 2
channel_width = 0.3 # Microns

# Loadstar parameters
alpha = 0.999
cutoff = 0.002

# MAGIK parameters
radius = 0.08 # Value between 0 and 1 where 1 is the full image hight
traj_min = 6
n_frames = 7

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

loadstar, magik = utils.load_models(loadstar_weight_name, magik_weight_name)

#### Test loadstar parameters

In [ ]:
utils.test_loadstar(tiff_path, loadstar, alpha, cutoff, 1)

### Extract metadata

In [ ]:
tiffpage = tifffile.TiffFile(tiff_path).pages[0]
for tag in tiffpage.tags.values():
    print(tag.name, tag.value)

In [ ]:
width = tiffpage.tags.get('ImageWidth').value # Pixels
height = tiffpage.tags.get('ImageLength').value # Pixels
x_resolution = tiffpage.tags.get('XResolution').value # Pixels per micron
y_resolution = tiffpage.tags.get('YResolution').value # Pixels per micron

### Calculated paramters

In [ ]:
# Calculated parameters
x_resolution = x_resolution[0] / 1000000 # Pixels per micron
y_resolution = y_resolution[0] / 1000000 # Pixels per micron

pixel_size_x = 1/x_resolution # Microns per pixel
pixel_size_y = 1/y_resolution # Microns per pixel

channel_width_pixels = channel_width * x_resolution # Pixels
channel_width_norm = channel_width_pixels / width

radius_ratio = radius / channel_width_norm

print(f"Channel width: {channel_width_pixels} pixels")
print(f"Radius ratio: {radius_ratio} ratio")

## Detect

In [ ]:
detections  = utils.detect(tiff_path, loadstar, pages_batch_size, alpha, cutoff)
utils.save_detections(detections, f"{output_path}/detections.csv", full=True)

## Track

In [ ]:
detections = utils.load_detections(f"{output_path}/detections.csv")

detections = utils.warp_axis(detections, radius_ratio, axis_name='centroid-0')
tracks = track.predict_trajectories(magik, detections, radius, n_frames, traj_min)
tracks = utils.unwarp_axis(tracks, radius_ratio, axis_name='x')

# Convert to pixel values
tracks['x'] *= width
tracks['y'] *= height
track.save_trajectories(tracks, f"{output_path}/tracks.csv")

### Plot track

In [ ]:
track.plot_trajectories(tracks, tiff_path, output_path, pages_batch_size, stop=50)

## Further analyze the tracks

### Change x and y units

#### Add axis for lengths in microns

In [ ]:
tracks = track.load_trajectories(f"{output_path}/tracks.csv")

tracks['x_microns'] = tracks['x'] * pixel_size_x
tracks['y_microns'] = tracks['y'] * pixel_size_y

#### Alt. replace x and y with micron versions

In [ ]:
tracks = track.load_trajectories(f"{output_path}/tracks.csv")
tracks['x'] = tracks['x'] * pixel_size_x
tracks['y'] = tracks['y'] * pixel_size_y

In [ ]:
tracks.head(20)

### Remove still particles

In [ ]:
still_cuttoff = 40

print(f'Number of objects before: {len(tracks.entity.unique())}')

tracks_stills_removed = track.remove_still_objects(tracks, still_cuttoff)

print(f'Number of objects after: {len(tracks_stills_removed.entity.unique())}')

In [ ]:
track.plot_trajectories(tracks_stills_removed, tiff_path, output_path, pages_batch_size, stop=50)

### Track length and distance

In [ ]:
tracks = tracks_stills_removed
tracks = track.distance_traveled(tracks)
tracks.head(20)

In [ ]:
tracks = track.count_appearances(tracks)
track.save_trajectories(tracks, f"{output_path}/tracks.csv")

tracks.head(50)

### Extract intensity

In [ ]:
tracks = utils.detection_intensity(tiff_path, tracks, pages_batch_size, mode='mean', kernel_size=3)
track_intensities = utils.track_intensity(tracks, mode='mean')

In [ ]:
tracks.head(10)

In [ ]:
print(track_intensities)